In [1]:
%config Completer.use_jedi = False



In [2]:
import torch
from tape import TAPETokenizer

In [3]:
from tape import ProteinBertForValuePredictionFragmentationProsit

In [4]:
import pickle as pkl

In [5]:
import pandas as pd

In [6]:
import numpy as np

In [7]:
from tqdm import tqdm

In [8]:
from prosittransformer.DataHandler import pad_sequences

In [9]:
from prosittransformer.utils import hdf5Loader
from prosittransformer.utils import cleanTapeOutput

In [46]:
from tape.datasets import PrositFragmentationDataset
from torch.utils.data import DataLoader

In [49]:
dataset = PrositFragmentationDataset("/sdd/PrositToTapeDataConverter/LMDB/", split="test")

In [52]:
loader = DataLoader(dataset, 
                    num_workers=6,
                    collate_fn=dataset.collate_fn,
                    batch_size=2)

In [65]:
b = next(iter(loader))

In [66]:
b = {name: tensor.cuda(device=torch.device('cuda:0'), non_blocking=True)
                     for name, tensor in b.items()}

In [79]:
dataset[1]

(array([ 2, 28,  8, 15, 25, 17, 15,  5, 25, 14,  3]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 array([ 0.13968435,  0.        , -1.        ,  0.        ,  0.        ,
        -1.        ,  0.14424464,  0.        , -1.        ,  0.8619605 ,
         0.        , -1.        ,  0.1845281 ,  0.        , -1.        ,
         0.18278196,  0.        , -1.        ,  0.11517897,  0.        ,
        -1.        ,  0.00940706,  0.        , -1.        ,  0.6255413 ,
         0.        , -1.        ,  0.        ,  0.        , -1.        ,
         0.72780836,  0.        , -1.        ,  0.01233532,  0.        ,
        -1.        ,  1.        ,  0.04024543, -1.        ,  0.01418118,
         0.        , -1.        ,  0.78416973,  0.        , -1.        ,
         0.        ,  0.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.   

In [80]:
OUT = model(**b)[1].cpu().detach().numpy()

In [81]:
OUT[1]

array([ 2.52864778e-01, -1.48597881e-02, -9.85823199e-03,  9.00416169e-04,
       -1.48429275e-02, -9.86088440e-03,  2.58889228e-01, -1.67866889e-02,
       -9.96968430e-03,  1.27201343e+00, -1.61841735e-02, -9.46859922e-03,
        3.22919071e-01, -1.57978255e-02, -9.07052495e-03,  2.76647329e-01,
       -1.38033116e-02, -9.17060953e-03,  1.85729593e-01, -1.34213371e-02,
       -9.44500789e-03,  1.37496516e-02, -1.26059288e-02, -1.10700261e-02,
        8.45821619e-01, -9.83877294e-03, -9.81847849e-03,  2.73358636e-03,
       -1.52619109e-02, -8.54323618e-03,  9.64756012e-01, -1.19160330e-02,
       -2.83062998e-02,  7.46198371e-03, -1.57763287e-02, -8.96913558e-03,
        1.29648697e+00,  2.61553861e-02,  3.04961428e-02, -8.25246796e-04,
       -1.42647419e-02,  3.30376439e-03,  9.29014444e-01, -1.47014307e-02,
        7.59038702e-02, -1.48373926e-02, -1.65109988e-02,  4.16598976e-01,
        1.03467786e+00,  4.25279647e-01,  2.39662990e-01,  4.71133441e-02,
        1.81184523e-02,  

In [12]:
prosit_pp = hdf5Loader.from_hdf5("/sdd/berzelius/final_results/delta_0.15/torchResult.hdf5")

Instructions for updating:
Training with HDF5Matrix is not optimized for performance. Instead, we recommend using https://github.com/tensorflow/io to load your HDF5 data into a tf.data Dataset and passing that dataset to Keras.


/home/ekvall/anaconda3/envs/tapeConvert2/lib/python3.6/site-packages/tensorflow/python/keras/utils/io_utils.py:133: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  f = h5py.File(datapath)


In [13]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        X = min(len(lst), i+n)
        yield lst[i:X]

In [14]:
collision_energy_aligned_normed = getVals(prosit_pp, "collision_energy_aligned_normed")
precursor_charge_onehot = getVals(prosit_pp, "precursor_charge_onehot")

NameError: name 'getVals' is not defined

In [ ]:
sequence_integer = getVals(prosit_pp, "sequence_integer")

In [ ]:
collision_energy_aligned_normed[0]

In [ ]:
precursor_charge_onehot[0]

In [ ]:
sequence_integer[0]

In [ ]:
seq = sequence_integer[0]
seq = seq[np.nonzero(seq)]

In [ ]:
peptide = "".join([ALPHABET_S[s] for s in seq])

In [64]:
model = ProteinBertForValuePredictionFragmentationProsit.from_pretrained("/sdd/berzelius/final_results/delta_0.15/")
model = model.to(torch.device('cuda:0'))

In [82]:
ix = 1

In [83]:
collision_energy = np.array([dataset[ix][3]])
charge = dataset[ix][4][None,:]
input_ids = dataset[ix][0][None,:]
input_mask = dataset[ix][1][None,:]

In [84]:
toy_data1 = {
    'collision_energy': torch.FloatTensor(collision_energy.astype(np.float32)),
    'charge': torch.FloatTensor(charge.astype(np.float32)),
    'input_ids' : torch.from_numpy(input_ids.astype(np.int64)),
    'input_mask' : torch.from_numpy(input_mask.astype(np.int64))
        }

toy_data1 = {name: tensor.cuda(device=torch.device('cuda:0'), non_blocking=True)
                     for name, tensor in toy_data1.items()}

In [85]:
model(**toy_data1)[0].cpu().detach().numpy()

array([[ 2.52864689e-01, -1.48597881e-02, -9.85823199e-03,
         9.00415704e-04, -1.48429256e-02, -9.86088254e-03,
         2.58889228e-01, -1.67866908e-02, -9.96968336e-03,
         1.27201343e+00, -1.61841717e-02, -9.46859829e-03,
         3.22919011e-01, -1.57978237e-02, -9.07052495e-03,
         2.76647329e-01, -1.38033116e-02, -9.17060860e-03,
         1.85729593e-01, -1.34213362e-02, -9.44500789e-03,
         1.37496516e-02, -1.26059279e-02, -1.10700252e-02,
         8.45821619e-01, -9.83877294e-03, -9.81847756e-03,
         2.73358263e-03, -1.52619118e-02, -8.54323618e-03,
         9.64756012e-01, -1.19160376e-02, -2.83063017e-02,
         7.46197999e-03, -1.57763269e-02, -8.96913558e-03,
         1.29648685e+00,  2.61553898e-02,  3.04961428e-02,
        -8.25254247e-04, -1.42647410e-02,  3.30376625e-03,
         9.29014385e-01, -1.47014335e-02,  7.59038702e-02,
        -1.48373973e-02, -1.65109932e-02,  4.16599005e-01,
         1.03467786e+00,  4.25279588e-01,  2.39663020e-0

In [22]:
CHARGES = [1, 2, 3, 4, 5, 6]
def get_precursor_charge_onehot(charges):
    array = np.zeros([len(charges), max(CHARGES)], dtype=int)
    for i, precursor_charge in enumerate(charges):
        array[i, precursor_charge - 1] = 1
    return array

In [23]:
df = pd.read_csv("/sdd/prosittransformer/data/prosit_input.csv", sep=",")

In [24]:
chunk = df.iloc[0,:].values

In [25]:
collision_energy1 = np.array([chunk[1] / 100.0])

In [26]:
charge1 = get_precursor_charge_onehot([chunk[2]])

In [27]:
peptide_sequences1 = chunk[0]

In [40]:
tokenizer = TAPETokenizer()
def TokenizePeptides(peptides):
    input_ids = np.array([tokenizer.encode(p) for p in peptides])
    return input_ids, np.ones_like(input_ids)

In [41]:
input_ids1, input_mask1 = TokenizePeptides([peptide_sequences1])

In [42]:
toy_data2 = {
    'collision_energy': torch.FloatTensor(collision_energy1.astype(np.float32)),
    'charge': torch.FloatTensor(charge1.astype(np.float32)),
    'input_ids' : torch.from_numpy(input_ids1.astype(np.int64)),
    'input_mask' : torch.from_numpy(input_mask1.astype(np.int64))
        }

toy_data2 = {name: tensor.cuda(device=torch.device('cuda:0'), non_blocking=True)
                     for name, tensor in toy_data2
             .items()}

In [43]:
toy_data2

{'collision_energy': tensor([0.3171], device='cuda:0'),
 'charge': tensor([[0., 0., 1., 0., 0., 0.]], device='cuda:0'),
 'input_ids': tensor([[ 2, 13, 17, 13,  8, 12, 14, 10, 12, 21, 12, 15,  3]], device='cuda:0'),
 'input_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [44]:
toy_data1

{'collision_energy': tensor([0.3171], device='cuda:0'),
 'charge': tensor([[0., 0., 1., 0., 0., 0.]], device='cuda:0'),
 'input_ids': tensor([[ 2, 13, 17, 13,  8, 12, 14, 10, 12, 21, 12, 15,  3]], device='cuda:0'),
 'input_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [45]:
model(**toy_data2)[0].cpu().detach().numpy()

array([[ 5.32360077e-02, -9.01104324e-03, -8.20380170e-03,
        -6.52382709e-03, -8.96532089e-03, -8.20204709e-03,
         1.05591387e-01, -1.01910308e-02, -8.30773823e-03,
         2.46346027e-01, -9.67752188e-03, -7.97981024e-03,
         1.16022319e-01, -9.50901583e-03, -8.15635175e-03,
        -2.01027095e-03, -9.89710353e-03, -8.71482491e-03,
         2.18849808e-01,  9.43607185e-03, -8.53640866e-03,
         4.24470752e-04, -7.14800786e-03, -8.31007119e-03,
         4.38355148e-01,  1.12281479e-02, -8.66800174e-03,
        -5.71816415e-03, -7.53244245e-03, -8.58819112e-03,
         2.22736239e-01,  5.47584519e-02, -1.00560375e-02,
        -3.55595350e-03, -7.87417777e-03, -6.74730027e-03,
         6.34075552e-02,  6.80070698e-01, -2.50562653e-03,
        -9.92925279e-03, -7.86461774e-03, -7.83468969e-03,
         4.08373401e-02,  3.62772614e-01, -3.57127725e-03,
        -9.26792808e-03, -7.70734623e-03, -8.01162329e-03,
        -1.77022927e-02,  3.71635973e-01, -5.36109600e-0

In [ ]:
collision_energy, charge, peptide_sequences = np.hstack(list(chunk[1] / 100.0)), get_precursor_charge_onehot(list(chunk[2])), list(chunk[0])



In [ ]:
DATA[0]

In [ ]:
prosit_pp = hdf5Loader.from_hdf5("/sdd/berzelius/final_results/delta_0.15/torchResult.hdf5")
prosit_vanilla = hdf5Loader.from_hdf5("/sdd/PrositToTapeDataConverter/hdf5/hcd/HDF5/prediction_hcd_ho.hdf5")

In [ ]:
ALPHABET = {
    "A": 1,
    "C": 2,
    "D": 3,
    "E": 4,
    "F": 5,
    "G": 6,
    "H": 7,
    "I": 8,
    "K": 9,
    "L": 10,
    "M": 11,
    "N": 12,
    "P": 13,
    "Q": 14,
    "R": 15,
    "S": 16,
    "T": 17,
    "V": 18,
    "W": 19,
    "Y": 20,
    "M(ox)": 21,
}
ALPHABET_S = {integer: char for char, integer in ALPHABET.items()}

In [ ]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        X = min(len(lst), i+n)
        yield lst[i:X]

In [ ]:
prosit_pp.keys()

In [ ]:
def getVals(data, key):
    sa_list = list()
    M = data[key]
    for b in tqdm(chunks(M, 1000),total =int(len(M) / 1000)):
        sa_list.append(b)
    all_sa = [s for sa in sa_list for s in sa]
    return np.array(all_sa)

In [ ]:
prosit_pp.keys()

In [ ]:
collision_energy_aligned_normed = getVals(prosit_pp, "collision_energy_aligned_normed")
precursor_charge_onehot = getVals(prosit_pp, "precursor_charge_onehot")


In [ ]:
sequence_integer = getVals(prosit_pp, "sequence_integer")

In [ ]:
pp_intensities_pred = getVals(prosit_pp, "intensities_pred")
pp_intensities_raw = getVals(prosit_pp, "intensities_raw")

In [ ]:
vanilla_intensities_pred = getVals(prosit_vanilla, "intensities_pred")
vanilla_intensities_raw = getVals(prosit_vanilla, "intensities_raw")

In [ ]:
X = np.load("/sdd/prosittransformer/data/delta_0.15/predicted_spectra.npy")

In [ ]:
X[1]

In [ ]:
pp_intensities_pred[1]

In [ ]:
charge = np.where(precursor_charge_onehot)[1]

In [ ]:
mask = [True if 0 < c and c <3 else False for c in charge]

In [ ]:
mask2 = [True if 21 not in seq else False for seq in sequence_integer]

In [ ]:
sum(mask2)

In [ ]:
MASK = [all([m1, m2]) for m1, m2 in zip(mask, mask2)]

In [ ]:
pp_sa = cleanTapeOutput.masked_spectral_distance(pp_intensities_raw[MASK], pp_intensities_pred[MASK])
1 - np.median(pp_sa)

In [ ]:
vanilla_sa = cleanTapeOutput.masked_spectral_distance(vanilla_intensities_raw[MASK], vanilla_intensities_pred[MASK])
1 - np.median(vanilla_sa)

In [ ]:
modified_sequence = ["".join([ALPHABET_S[s] for s in seq[np.nonzero(seq)]]) for seq in sequence_integer[MASK]]

In [ ]:
charge_val = charge[MASK] + 1
    

In [ ]:
col_energy = collision_energy_aligned_normed[MASK] * 100

In [ ]:
len(col_energy)

In [ ]:
len(charge_val)

In [ ]:
len(modified_sequence)

In [ ]:
df = pd.DataFrame([[p, col, c] for p, c, col in zip(modified_sequence, charge_val, col_energy)], 
             columns=["modified_sequence","collision_energy","precursor_charge"])

In [ ]:
df.to_csv("/sdd/prosittransformer/data/prosit_input.csv", index=False, sep=",")

In [ ]:
'collision_energy_aligned_normed', 'sequence_integer', 'precursor_charge_onehot', 'masses_pred', 'intensities_pred', 'iRT'

In [ ]:
all_results = pkl.load(open("/sdd/prosittransformer/data/all_result.pkl", "rb"))

In [ ]:
all_results.keys()

In [ ]:
all([p1==p2 for p1, p2 in zip(modified_sequence, all_results['modified_sequence'])])

In [ ]:
all_results['modified_sequence']

In [ ]:
all_results['iRT']

In [ ]:
0 < charge[0] and charge[0] <3

In [ ]:
pp_sa = cleanTapeOutput.masked_spectral_distance(pp_intensities_raw, pp_intensities_pred)
1 - np.median(pp_sa)

In [ ]:
vanilla_sa = cleanTapeOutput.masked_spectral_distance(vanilla_intensities_raw, vanilla_intensities_pred)
1 - np.median(vanilla_sa)

In [ ]:
1 - np.median(pp_sa)

In [ ]:
np.median(all_sa)